In [ ]:
# Fine-tuner FLAN-T5 avec PEFT et RLHF (PPO) --> output moins toxiques
# On veut diminuer les contenus haineux

In [ ]:
# 1 Set up Kernel and required dependencies

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade pip setuptools wheel
#!{sys.executable} -m pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1
!{sys.executable} -m pip install --disable-pip-version-check torch torchdata
!{sys.executable} -m pip install transformers==4.27.2 datasets==2.11.0 \
    evaluate==0.4.0 rouge_score==0.1.2 loralib==0.1.1 peft==0.3.0 trl==0.4.4
# Le dernier module est nouveau par rapport à Week2, pour faire de la PPO
# Cette dernière library suit l'architecture habituelle de HuggingFace : Trainer et training arguments

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 262.6 kB/s eta 0:00:08
   -- ------------------------------------- 0.1/2.1 MB 656.4 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 958.4 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/2.1 MB 1.4 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.1 MB 1.6 MB/s eta 0:00:02
   ------------ --------------------------- 0.7/2.1 MB 1.9 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.1 MB 2.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.1 MB 2.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.4/2.1 MB 2.7 MB/s eta 0:00:01
   ------------------------

In [3]:
# Importation des composants nécessaires
from datasets import load_dataset
# AutoModelForSeq2SeqLM pour accéder à FLAN-T5
# AutoModelForSequenceClassification pour charger un modèle de classification binaire des messages (haineux ou non)
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl = transformers reinforcement learning library
from trl import PPOConfig, PPOTrainer, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
# LengthSampler au cas où les inputs sont plus longs que la fenêtre d'input
from trl.core import LengthSampler
import torch
import time
import evaluate
import pandas as pd
import numpy as np

# Pour voir joliment la progression des calculs
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# 2 Load FLAN-T5 model, prepare Reward model et toxicity evaluator

In [ ]:
# 2.1 Load data et FLAN-T5 model fine-tuned with summarization instruction

In [4]:
model_name='google/flan-t5-small'
huggingface_dataset_name="knkarthick/dialogsum"
dataset_original=load_dataset(huggingface_dataset_name)
dataset_original

Found cached dataset csv (C:/Users/V/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [6]:
# On crée une fonction build_dataset pour mettre en forme les données en une seule commande
# On ne prend qu'une partie des données, on ne prend que celles d'une certaine longueur, 
# on structure les prompts (instruction, dialogue) et on les tokenise
# On sauvegarde les token_ids (input_ids) et les prompts décodés (query)

def build_dataset(model_name, dataset_name, input_min_text_length,input_max_text_length):
    # Load dataset (seule la partie train est utile dans cet exercice)
    dataset=load_dataset(dataset_name, split='train')
    # Filtre sur la longueur des dialogues
    dataset=dataset.filter(lambda x: len(x['dialogue']) > input_min_text_length and len(x['dialogue']) <= input_max_text_length,batched=False)
    # Importer le tokenizer (le même que celui du modèle choisi !)
    tokenizer=AutoTokenizer.from_pretrained(model_name, device_map='auto')
    
    def tokenize(sample):
        prompt=f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample['input_ids']=tokenizer.encode(prompt)
        # Il faut absolument appeler "query" la version décodée du prompt structuré, car c'est la variable d'appel de PPO
        sample['query']=tokenizer.decode(sample['input_ids'])
        return sample
    
    # Tokenisation de chaque dialogue du dataset
    dataset=dataset.map(tokenize,batched=False)
    dataset.set_format(type="torch")
    
    # Split the dataset into train and test parts
    dataset_splits=dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)
    
    return dataset_splits

dataset=build_dataset(model_name=model_name, dataset_name=huggingface_dataset_name, input_min_text_length=200,input_max_text_length=500)
print(dataset)

Found cached dataset csv (C:/Users/V/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

C:\Users\V\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\V\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Map:   0%|          | 0/3421 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2736
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 685
    })
})


In [9]:
# On crée une fonction pour montrer les différents paramètres du modèles, en particulier les trainables
def print_number_of_trainable_model_parameters(model):
    trainable_model_params=0
    all_model_params=0
    for _,param in model.named_parameters():
        all_model_params+=param.numel()
        if param.requires_grad:
            trainable_model_params+=param.numel()
    return f"trainable model parameters : {trainable_model_params} \nall model parameters : {all_model_params} \npercentage of trainable model parameters : {trainable_model_params/all_model_params}"

In [17]:
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters : 247577856 
all model parameters : 247577856 
percentage of trainable model parameters : 1.0


In [ ]:
# On récupère les adapters de LoRA estimés en week2 et on les fusionne avec le modèle original
# C'est ce modèle avec adapters qui sera entraîné par RLHF (d'où "is_trainable=True" ci-dessous)

lora_config=LoraConfig(r=32,lora_alpha=32,target_modules=["q","v"],
                      lora_dropout=0.05, bias="none",task_type=TaskType.SEQ2SEQLM)

model=AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

peft_model=PeftModel.from_pretrained(model,'./peft-dialogue-summary-checkpoint/',
                                    lora_config=lora_config, torch_dtype=torch.bfloat16,
                                    device_map="auto",is_trainable=True)

print(f'PEFT Model parameters to be updated : \n{print_number_of_trainable_model_parameters(peft_model)}\n')

In [ ]:
# On prépare le modèle avec PPO
ppo_model=AutoModelForSeq2SeqLMWithValueHead(peft_model, torch_dtype=torch.bfloat16,is_trainable=True)
print(f'PPO model parameters to be updated (ValueHead + 769 parameters) : {print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)
# Il y a 768 dimensions pour le ValueHaead, plus le terme de biais
# Le nombre de paramètres qui seront mis à jour (=ValueHead) est égal à (n+1)*m où n=nombre d'inputs (ici 768)
# et m=nombre d'outputs (ici 1)

In [ ]:
# On stocke le modèle avant detoxification comme référence, pour le calcul de la divergence KL
reference_model=create_reference_model(ppo_model)
print(f'Reference model parameters to be updated : {print_number_of_trainable_model_parameters(reference_model)}\n')

In [ ]:
# 2.2 Prepare reward model

In [ ]:
# On utilise un reward model sur étagère, qui donne un niveau de haine (logit)
# hate =1, not_hate=0
toxicity_model_name='facebook/roberta-hate-speech-dynabench-r4-target'
toxicity_tokenizer=AutoTokenizer.from_pretrained(toxicity_model_name, device_map='auto')
toxicity_model=AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map='auto'))
print(toxicity_model.config.id2label)

In [ ]:
# Application à un simple texte
non_toxic_text="I want to kiss you"
toxicity_input_ids=toxicity_tokenizer(non_toxic_text,return_tensors='pt').input_ids

logits=toxicity_model(input_ids=toxicity_input_ids).logits
print(f'Logits [not hate, hate] de la phrase {non_toxic_text}:\n {logits.tolist()[0]}')

probas=logits.softmax(dim=-1).tolist[0]
print(f'Probas [not hate, hate] de la phrase {non_toxic_text}:\n {probas}')

# Reward = not hate logit
not_hate_index=0
not_hate_reward=(logits[:,not_hate_index]).tolist()
print(f'Reward: {not_hate_reward}')

In [ ]:
# Application à un autre texte
toxic_text="You are disgusting and terrible and I damn hate you"
toxicity_input_ids=toxicity_tokenizer(toxic_text,return_tensors='pt').input_ids

logits=toxicity_model(input_ids=toxicity_input_ids).logits
print(f'Logits [not hate, hate] de la phrase {toxic_text}:\n {logits.tolist()[0]}')

probas=logits.softmax(dim=-1).tolist[0]
print(f'Probas [not hate, hate] de la phrase {toxic_text}:\n {probas}')

# Reward = not hate logit
not_hate_index=0
not_hate_reward=(logits[:,not_hate_index]).tolist()
print(f'Reward: {not_hate_reward}')

In [ ]:
# Quelques définitions utilisant le pipeline de HuggingFace pour simplifier le code du reward model
# Plus besoin d'utiliser model.generate ou tokenizer, 
# c'est déjà dans le pipeline "sentiment-analysis" qui est un classificateur binaire de textes

device=0 if torch.cuda.is_available() else "cpu"

sentiment_pipe=pipeline("sentiment-analysis",model=toxicity_model_name,device=device)

# top_k=None pour avoir tout le monde, function_to_apply=None pour récupérer les logits sans transformation
reward_logits_kwargs={"top_k": None, "function_to_apply": None, "batch_size"=16}
reward_probas_kwargs={"top_k": None, "function_to_apply": "softmax", "batch_size"=16}

print("Reward model output for non-toxic text : ")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probas_kwargs))
print("Reward model output for toxic text : ")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probas_kwargs))

In [ ]:
# 2.3 Evaluate toxicity

In [ ]:
# On définit une métrique de toxicité comprise entre 0 et 1 : on prend les probas associées aux logits du modèle précédent

toxicity_evaluator=evaluate.load("toxicity", toxicity_model_name, module_type="measurement",toxic_label='hate')

In [ ]:
# Calculs de métrique de toxicité

toxicity_score=toxicity_evaluator.compute(predictions=[non_toxic_text])

print(f'Toxicity score for non_toxic_text {non_toxic_text}')
print(toxicity_score['toxicity'])

toxicity_score=toxicity_evaluator.compute(predictions=[toxic_text])

print(f'\nToxicity score for toxic_text {toxic_text}')
print(toxicity_score['toxicity'])

In [ ]:
# On rassemble dans une fonction evaluate_toxicity les étapes suivantes : passer en revue le dataset de test,
# utiliser le bon tokenizer, charger le PEFT model et le toxicity evaluator
# Permet d'avaluer la toxicité moyenne d'un ensemble d'outputs (ici des résumés de dialogues)

def evaluate_toxicity(model, toxicity_evaluator, tokenizer, dataset, num_samples):
    max_nex_tokens=100
    
    toxicities=[]
    input_texts=[]
    for i,sample in tqdm(enumerate(dataset)):
        input_text=sample["query"]
        if i>num_samples:
            break
        
        input_ids=tokenizer(input_text, return_tensors='pt',padding=True).input_ids
        generation_config=GenerationConfig(max_new_tokens=max_new_tokens,tok_k=0.0, tok_p=1.0,do_sample=True)
        response_token_ids=model.generate(input_ids=input_ids,generation_config=generation_config)
        
        generated_text=tokenizer.decode(response_token_ids[0], skip_special_tokens=True)
        
        toxicity_score=toxicity_evaluator.compute(predictions=[(input_text+" "+generated_text)])
        
        toxicities.extend(toxicity_score["toxicity"])
        
        # On calcule des stats de toxicité
        mean=np.mean(toxicities)
        std=np.std(toxicities)
        
        return mean,std

In [ ]:
# On calcule ici le niveau de toxicité du modèle avant detoxification

tokenizer=AutoTokenizer.from_pretrained(model_name,device_map='auto')

mean_before_detoxification,std_before_detoxification=evaluate_toxicity(model=ref_model,toxicity_evaluator=toxicity_evaluator
                                                                      tokenizer=tokenizer, dataset=dataset['test'],
                                                                      num_samples=10)

print(f'Moyenne et écart-type de la toxicité avant détoxification : [{mean_before_detoxification},{std_before_detoxification}]')
# Rq sur la performance : le lab de AWS le fait en 25 secondes

In [ ]:
# 3 Fine-tuning pour diminuer la toxicité des outputs

In [ ]:
# 3.1 Initialiser PPOTrainer

In [ ]:
# Configuration de paramètres de PPO

learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config=PPOConfig(model_name=model_name,learning_rate=learning_rate,max_ppo_epochs=max_ppo_epochs,
                mini_batch_size=mini_batch_size,batch_size=batch_size)

def collator(data):
    return dict((key,[d[key] for d in data] for key in data[0]))

# Exemple d'utilisation de collator
test_data=[{"key1":"value1","key2":"value2","key3":"value3"}]
print(f'Collator input : {test_data}')
print(f'Collator output : {collator(test_data)}')

ppo_trainer=PPOTrainer(config=config,model=ppo_model,ref_model=ref_model,
                       tokenizer=tokenizer,dataset=dataset['train'],data_collator=collator)

In [ ]:
# 3.2 Fine-tune le modèle

In [ ]:
# On réalise un grand nombre de fois les trois étapes suivantes :
# on récupère les réponses du LLM
# on calcule les rewards
# on optimise la règle PPO en fonction des infos ci-dessus

In [ ]:
# Cette cellule dure 20 à 30 minutes dans le lab de AWS

output_min_length=100
output_max_length=400

output_length_sampler=LengthSampler(output_min_length,output_max_length)

generation_kwargs={"min_length":0.5,"top_k":0.0,"top_p":1.0,"do_sample":True}

reward_kwargs={"top_k":None,"function_to_supply":None, "batch_size":16}

max_ppo_steps=10

# Pour chaque dialogue du dataset, le modèle fournit une réponse, on calcule le reward du prompt+réponse

for step,batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step>=max_ppo_steps:
        break
    
    prompt_tensors=batch['input_ids']
    
    #Réponse de FLan-T5 avec PEFT
    summary_tensors=[]
    
    for prompt_tensor in prompt_tensors:
        max_new_tokens=output_length_sampler
        generation_kwargs["max_new_tokens"]=max_new_tokens
        summary=ppo_trainer.generate(prompt_tensor,**generation_kwargs)
        
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])
        
    # Il faut utiliser le terme "response" pour que ce soit reconnu par l'optimiseur
    batch["response"]=[tokenizer.decode((r.squeeze()) for r in summary_tensors)
                       
    # Calcul des rewards
    query_response_pairs=[q+r for q,r in zip(batch["query"],batch["response"])]
    rewards=sentiment_pipe(query_response_pairs, **reward_kwargs)
    reward_tensors=[torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]
    
    # Étape PPO (en n'entraînant que les paramètres PEFT/LoRA)
    stats=ppo_trainer.step(prompt_tensors, summary_tensors,reward_tensors)
    ppo_trainer.log_stats(stats,batch,reward_tensors)
                       
    print(f'objective/kl : {stats["objective/kl"]}')
    print(f'ppo/returns/mean : {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean : {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))
    

In [ ]:
# 3.3 Evaluate the model quantitatively

In [ ]:
mean_after_detoxification,std_after_detoxification=evaluate_toxicity(model=ppo_model,toxicity_evaluator=toxicity_evaluator
                                                                      tokenizer=tokenizer, dataset=dataset['test'],
                                                                      num_samples=10)

print(f'Moyenne et écart-type de la toxicité après détoxification : [{mean_after_detoxification},{std_after_detoxification}]')

In [ ]:
# 3.4 Evaluate the Model qualitatively

In [ ]:
# On regarde à la main quelques exemples

batch_size=20
compare_results={}
df_batch=dataset['test'][0:batch_size]

compare_results["query"]=df_batch['query']
prompt_tensors=df_batch['input_ids']

summary_tensors_ref=[]
summary_tensors=[]

for i in tqdm(range(batch_size)):
    gen_len=output_length_sampler()
    generation_kwargs["max_new_tokens"]=gen_len
    
    summary=ref_model.generate(input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
                              **generation_kwargs).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)
    
    summary=ppo_model.generate(input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
                              **generation_kwargs).squeeze()[-gen_len:]
    summary_tensors.append(summary)
    
# On décode les réponses
compare_results["response_before"]=[tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"]=[tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Calcul des rewards sur les paires query+response
texts_before=[d+s for d,s in zip(compare_results["query"],compare_results["response_before"])]
rewards_before=sentiment_pipe(texts_before,**reward_kwargs)
compare_results["reward_before"]=[reward[not_hate_index]["score"] for reward in rewards_before]

texts_after=[d+s for d,s in zip(compare_results["query"],compare_results["response_after"])]
rewards_after=sentiment_pipe(texts_after,**reward_kwargs)
compare_results["reward_after"]=[reward[not_hate_index]["score"] for reward in rewards_after]

    

In [ ]:
pd.set_option('display.max_colwidth',500)
df_compare_results=pd.DataFrame(compare_results)
df_compare_results["reward_diff"]=df_compare_results["reward_after"]-df_compare_results["reward_before"]
df_compare_results_sorted=df_compare_results.sort_values(by=['reward_diff'],ascending=False).reset_index(drop=True)
df_compare_results_sorted